### Operações de pooling

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

x = torch.arange(0, 16).reshape(1,1,4,4)
x = x.float().requires_grad_(True)
y = F.max_pool2d(x, kernel_size=2, stride=1)

print(x)
print(y)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]], requires_grad=True)
tensor([[[[ 5.,  6.,  7.],
          [ 9., 10., 11.],
          [13., 14., 15.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)


In [3]:
# Por padrão, a função max_pool2d utiliza stride=kernel_size, o que efetivamente reduz
# o tamanho da imagem pela metade
y = F.max_pool2d(x, kernel_size=2)

print(x)
print(y)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]], requires_grad=True)
tensor([[[[ 5.,  7.],
          [13., 15.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)


In [4]:
# Outra operação de pooling é o average pooling
y = F.avg_pool2d(x, kernel_size=2)

print(x)
print(y)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]], requires_grad=True)
tensor([[[[ 2.5000,  4.5000],
          [10.5000, 12.5000]]]], grad_fn=<AvgPool2DBackward0>)


In [5]:
# A operação acima é equivalente a convoluir a imagem com o seguinte filtro de média:
w = torch.tensor([[1,1],[1,1]])/4
w = w.reshape(1,1,2,2)

y = F.conv2d(x, w, stride=2)
print(y)

tensor([[[[ 2.5000,  4.5000],
          [10.5000, 12.5000]]]], grad_fn=<ConvolutionBackward0>)


### Pooling adaptativo

Toda rede de classificação precisa que a saída tenha um tamanho padrão e fixo. Camadas de pooling adaptativo possibilitam fixar o tamanho da saída. O stride, tamanho e padding do kernel é ajustado automaticamente para que a saída tenha o tamanho desejado.

In [6]:
F.adaptive_avg_pool2d(x, output_size=(3,3))

tensor([[[[ 2.5000,  3.5000,  4.5000],
          [ 6.5000,  7.5000,  8.5000],
          [10.5000, 11.5000, 12.5000]]]], grad_fn=<AdaptiveAvgPool2DBackward0>)

### Camadas de pooling

In [7]:
pool1 = nn.MaxPool2d(kernel_size=2)
pool2 = nn.MaxPool2d(kernel_size=2)
pool3 = nn.AdaptiveAvgPool2d(output_size=(5,5))

x = torch.rand((1,3,224,224))
y1 = pool1(x)
y2 = pool2(y1)
y3 = pool3(y2)

print(y1.shape, y2.shape, y3.shape)

torch.Size([1, 3, 112, 112]) torch.Size([1, 3, 56, 56]) torch.Size([1, 3, 5, 5])
